In [65]:
from tqdm import tqdm
import numpy as np
import os

In [66]:
def read_fasta(fasta_file):
    lines = fasta_file.readlines()
    body = ""
    for l in lines:
        if l[0] == ">":
            if body != "":
                yield name.strip('\n'), body.strip('\n')            
            name = l[1:]
            body = ""
        else:
            body += l.strip('\n')
    yield name.strip('\n'), body.strip('\n')

In [67]:
with open('data/yeast.fasta', 'r') as ff:
    yeast_dset = {}
    for name, seq in read_fasta(ff):
        yeast_dset[name] = seq

In [68]:
yeast_dset['Q9ZZX9']

'MYYIMFLYNMLLIIILIFYSIVGVPIIIFNNNYYWDPDIFLFIIYYFIKFIIIFNLYLYYMINYIVYTPSGSPPGRGTYILLYNMLYSYNMFIDYVMKFITCVTYMYLMFWLLSPTPSPYYVSEVPVS'

In [86]:
with open('data/human.fasta', 'r') as ff:
    human_dset = {}
    for name, seq in read_fasta(ff):
        if 'U' in seq or 'X' in seq:
            if name == "P80667":
                print(seq)
            continue
        human_dset[name] = seq

In [89]:
human_dset['P04786']

KeyError: 'P04786'

In [59]:
with open("data/human_ct/ignore.txt", "w+") as fff:
    with open('data/human.fasta', 'r') as ff:
        for name, seq in read_fasta(ff):
            if 'U' in seq or 'X' in seq:
                fff.write("{}\n".format(name))

In [90]:
def parse_splits_file(sp_file, label):
    data = []
    with open(sp_file, 'r') as f:
        lines = f.readlines()
        for l in lines:
            if label == 'neg':
                x,y = l.split(" ")
            else:
                x,y,_ = l.split(" ")
            if x == "P04786" or y == "P04786":
                import pdb; pdb.set_trace()
                print("heelo")
            data.append((x.strip('\n'), y.strip('\n'), label))
    return data

In [91]:
diff = ["C1", "C2", "C3"]
mode = ["train", "train", "test"]
splits = 10
pos_test_samples = [set(), set(), set()]
neg_test_samples = [set(), set(), set()]
for i in range(3):
    for j in range(splits):
        pos = parse_splits_file('data/negativeSimTest/nonRed/{}/yeastCV/split_0/{}.{}.pos'.format(diff[i], j, mode[i]), 'pos')
        neg = parse_splits_file('data/negativeSimTest/nonRed/{}/yeastCV/split_0/{}.{}.neg'.format(diff[i], j, mode[i]), 'neg')
        pos_test_samples[i] = pos_test_samples[i].union(set(pos))
        neg_test_samples[i] = neg_test_samples[i].union(set(neg))

> <ipython-input-90-e35f5b9d3b4e>(12)parse_splits_file()
-> print("heelo")
(Pdb) !sp_file
'data/negativeSimTest/nonRed/C1/yeastCV/split_0/0.train.pos'
(Pdb) exit


BdbQuit: 

In [122]:
splits = 10
for j in range(splits-1):
    pos = parse_splits_file('data/negativeSimTest/nonRed/C1/humanCV/split_0/{}.train.pos'.format(j), 'pos')
    neg = parse_splits_file('data/negativeSimTest/nonRed/C2/humanCV/split_0/{}.test.pos'.format(j), 'pos')
    pos = set(pos)
    neg = set(neg)
    print(len(pos.intersection(neg))/len(pos.union(neg)), len(pos), len(neg), len(pos.union(neg)))

0.0 758 518 1276
0.0 758 515 1273
0.0 758 527 1285
0.0 758 517 1275
0.0 757 523 1280
0.0 758 520 1278
0.0 758 504 1262
0.0 758 511 1269
0.0 758 513 1271


In [104]:
# C1 - C2
c1_c2_int = len(pos_test_samples[0].intersection(pos_test_samples[1]))
total = len(pos_test_samples[0].union(pos_test_samples[1]))
print("* pos test overlap btwn C1 and C2: {}".format(c1_c2_int/total))
c1_c2_int = len(neg_test_samples[0].intersection(neg_test_samples[1]))
total = len(neg_test_samples[0].union(neg_test_samples[1]))
print("* neg test overlap btwn C1 and C2: {}".format(c1_c2_int/total))
# C1 - C3
c1_c3_int = len(pos_test_samples[0].intersection(pos_test_samples[2]))
total = len(pos_test_samples[0].union(pos_test_samples[2]))
print("* pos test overlap btwn C1 and C3: {}".format(c1_c3_int/total))
c1_c3_int = len(neg_test_samples[0].intersection(neg_test_samples[2]))
total = len(neg_test_samples[0].union(neg_test_samples[2]))
print("* neg test overlap btwn C1 and C3: {}".format(c1_c3_int/total))
# C2 - C3
c2_c3_int = len(pos_test_samples[1].intersection(pos_test_samples[2]))
total = len(pos_test_samples[1].union(pos_test_samples[2]))
print("* pos test overlap btwn C2 and C3: {}".format(c2_c3_int/total))
c2_c3_int = len(neg_test_samples[1].intersection(neg_test_samples[2]))
total = len(neg_test_samples[1].union(neg_test_samples[2]))
print("* neg test overlap btwn C2 and C3: {}".format(c2_c3_int/total))

print()
for i in range(3):
    print(" pos test size for C{}: {}".format(i+1, len(pos_test_samples[i])))
    print(" neg test size for C{}: {}".format(i+1, len(neg_test_samples[i])))

* pos test overlap btwn C1 and C2: 0.1828612405880244
* neg test overlap btwn C1 and C2: 0.00020738282870178348
* pos test overlap btwn C1 and C3: 0.0
* neg test overlap btwn C1 and C3: 0.0
* pos test overlap btwn C2 and C3: 0.0
* neg test overlap btwn C2 and C3: 2.1546616103940875e-05

 pos test size for C1: 1636
 neg test size for C1: 33393
 pos test size for C2: 1663
 neg test size for C2: 38952
 pos test size for C3: 746
 neg test size for C3: 7460


## Yeast 
* pos test overlap btwn C1 and C2: 0.1828612405880244
* neg test overlap btwn C1 and C2: 0.00020738282870178348
* pos test overlap btwn C1 and C3: 0.0
* neg test overlap btwn C1 and C3: 0.0
* pos test overlap btwn C2 and C3: 0.0
* neg test overlap btwn C2 and C3: 2.1546616103940875e-05

## Human
* pos test overlap btwn C1 and C2: 0.13919952913478517
* neg test overlap btwn C1 and C2: 1.1159344276930287e-05
* pos test overlap btwn C1 and C3: 0.0
* neg test overlap btwn C1 and C3: 0.0
* pos test overlap btwn C2 and C3: 0.0
* neg test overlap btwn C2 and C3: 0.0

100% overlap between C1/C2 train and C3 test for positive


In [13]:
train_i[0][0]

'P21524'

## Autocovariance featurization

In [35]:
physicochemical_properties = np.array([[0.62, -0.5, 0.007187, 8.1, 0.046, 1.181, 27.5],
                                      [0.29, -1, -0.03661, 5.5, 0.128, 1.461, 44.6],
                                      [-0.9, 3, -0.02382, 13, 0.105, 1.587, 40],
                                      [-0.74, 3, 0.006802, 12.3, 0.151, 1.862, 62],
                                      [1.19, -2.5, 0.037552, 5.2, 0.29, 2.228, 115.5],
                                      [0.48, 0, 0.179052, 9, 0, 0.881, 0],
                                      [-0.4, -0.5, -0.01069, 10.4, 0.23, 2.025, 79],
                                      [1.38, -1.8, 0.021631, 5.2, 0.186, 1.81, 93.5],
                                      [-1.5, 3, 0.017708, 11.3, 0.219, 2.258, 100],
                                      [1.06, -1.8, 0.051672, 4.9, 0.186, 1.931, 93.5],
                                      [0.64, -1.3, 0.002683, 5.7, 0.221, 2.034, 94.1],
                                      [-0.78, 2, 0.005392, 11.6, 0.134, 1.655, 58.7],
                                      [0.12, 0, 0.239531, 8, 0.131, 1.468, 41.9],
                                      [-0.85, 0.2, 0.049211, 10.5, 0.18, 1.932, 80.7],
                                      [-2.53, 3, 0.043587, 10.5, 0.291, 2.56, 105],
                                      [-0.18, 0.3, 0.004627, 9.2, 0.062, 1.298, 29.3],
                                      [-0.05, -0.4, 0.003352, 8.6, 0.108, 1.525, 51.3],
                                      [1.08, -1.5, 0.057004, 5.9, 0.14, 1.645, 71.5],
                                      [0.81, -3.4, 0.037977, 5.4, 0.409, 2.663, 145.5],
                                      [0.26, -2.3, 117.3, 6.2, 0.298, 2.368, 0.023599]])

mu = np.mean(physicochemical_properties, axis=0, keepdims=True)
sig = np.var(physicochemical_properties, axis=0, keepdims=True)
pp_normed = (physicochemical_properties - mu) / sig
print(pp_normed)
print("-----")
print(mu)
print("-----")
print(sig)

[[ 6.52927114e-01 -1.00003333e-01 -9.02148736e-03 -3.27183495e-02
  -1.43409980e+01 -3.15187416e+00 -2.82959477e-02]
 [ 3.05401392e-01 -2.33341111e-01 -9.08854103e-03 -4.10797055e-01
  -5.27770831e+00 -1.76773871e+00 -1.59466334e-02]
 [-9.47797424e-01  8.33361112e-01 -9.06895940e-03  6.79814596e-01
  -7.81985054e+00 -1.14487775e+00 -1.92686711e-02]
 [-7.79300104e-01  8.33361112e-01 -9.02207680e-03  5.78024175e-01
  -2.73556609e+00  2.14540995e-01 -3.38066447e-03]
 [ 1.25319882e+00 -6.33354445e-01 -8.97499823e-03 -4.54421521e-01
   1.26278152e+01  2.02380377e+00  3.52560790e-02]
 [ 5.05491960e-01  3.33344445e-02 -8.75836025e-03  9.81550486e-02
  -1.94252824e+01 -4.63487644e+00 -4.81559560e-02]
 [-4.21243300e-01 -1.00003333e-01 -9.04885724e-03  3.01735890e-01
   5.99613981e+00  1.02030556e+00  8.89643160e-03]
 [ 1.45328938e+00 -4.46681556e-01 -8.99937345e-03 -4.54421521e-01
   1.13291121e+00 -4.25127318e-02  1.93680724e-02]
 [-1.57966237e+00  8.33361112e-01 -9.00537960e-03  4.32609288e-0

In [40]:
def autocovariance(seq, pp, lag=30):
    """Compute and return normalized AC featurized numpy array for given sequence.
    Dimensions are 420x1
    """
    if len(seq) - lag < 1:
        return None
    num_pp = 7
    aa = {'A': 0, 'C': 1, 'D': 2, 'E': 3, 'F': 4, 'G': 5, 'H': 6, 'I': 7, 'K': 8, 'L': 9, 'M': 10, 'N': 11, 'P': 12,
        'Q': 13, 'R': 14, 'S': 15, 'T': 16, 'V': 17, 'W': 18, 'Y': 19}    
    def ac_vector(seq):
        ac = np.zeros((lag,num_pp))
        for l in range(lag):
            for j in range(num_pp):
                for i in range(len(seq) - l):
                    if not seq[i] in aa:
                        return None
                    if not seq[i+l] in aa:
                        return None
                    amino_acid_i = aa[seq[i]]
                    amino_acid_i_l = aa[seq[i+l]]
                    ac[l,j] += (pp[amino_acid_i, j] - pp[:,j].sum()) * (pp[amino_acid_i_l,j] - pp[:,j].sum())
                ac[l,j] *= 1 / (len(seq) - l)
        return ac
    x = ac_vector(seq)
    if x is None:
        return x
    return x.flatten()

### Save AV vectors in npy format

In [28]:
for k,v in yeast_dset.items():
    if os.path.exists("data/yeast_ac/{}.npy".format(k)):
        continue
    feature_vector = autocovariance(v, pp_normed)
    if feature_vector is None:
        print("{}".format(k))
    np.save("data/yeast_ac/" + k + ".npy", feature_vector)

P08521
P0C1Z1
P0CX86
P0CX87
Q3E741
Q3E775
Q3E7Z6
Q3E801
Q3E838
Q8TGJ2
Q8TGN3
Q8TGS6
Q8TGS7
Q8TGT6
Q8TGT8
Q8TGU0
Q8TGV0


In [88]:
k ='Q9Y6Z7'
feature_vector = autocovariance(human_dset[k], pp_normed, lag=20)
if feature_vector is None:
    print("{}".format(k))
np.save("data/human_ac/" + k + ".npy", feature_vector)

In [48]:
lengths = []
for k,v in tqdm(human_dset.items()):
    lengths.append(len(v))
print((np.array(lengths) < 20).sum())


100%|██████████| 20127/20127 [00:00<00:00, 1312460.26it/s]

40


In [51]:
with open("data/human_ac/ignore.txt", "w+") as f:
    for k,v in tqdm(human_dset.items()):
        if os.path.exists("data/human_ac/{}.npy".format(k)):
            continue
        feature_vector = autocovariance(v, pp_normed, 20)
        if feature_vector is None:
            f.write("{}\n".format(k))
        np.save("data/human_ac/" + k + ".npy", feature_vector)


  0%|          | 11/20127 [00:03<1:55:49,  2.89it/s]Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/pemami/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/pemami/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/pemami/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 20127/20127 [1:34:58<00:00,  3.53it/s]


In [40]:
# for k,v in human_dset.items():
#     print(v)
#     break
human_dset['M0R2V9']

'CLIPIGVPSLKMVCEMKKGDTLW'

In [31]:
feature_vector.shape

(210,)

## Conjoint Triad method

In [71]:
def get_cluster_rep(seq):
    aa_cluster = {
        'A': 1,
        'R': 5,
        'N': 4,
        'D': 6,
        'C': 7,
        'E': 6,
        'Q': 4,
        'G': 1,
        'H': 4,
        'I': 2,
        'L': 2,
        'K': 5,
        'M': 3,
        'F': 2,
        'P': 2,
        'S': 3,
        'T': 3,
        'W': 4,
        'Y': 3,
        'V': 1
    }
    new_seq = ""
    for i in range(len(seq)):
        new_seq += str(aa_cluster[seq[i]])
    return new_seq

In [72]:
def convert_triad(seq):
    cluster_seq = get_cluster_rep(seq)
    result = np.zeros((343,))
    for i in range(len(cluster_seq)-2):
        triad = cluster_seq[i:i+3]
        a = int(triad[2])
        b = int(triad[1])
        c = int(triad[0])
        idx = ((a-1) * 7 ** 0) + ((b-1) * 7 ** 1) + ((c-1) * 7 ** 2)
        result[idx] += 1
    return result

In [11]:
#convert_triad(yeast_ds)

AttributeError: 'dict' object has no attribute 'at'

In [80]:
# find max freq value in dataset
max_frq = 0
fvs = []
for k,v in human_dset.items():
    fv = convert_triad(v)
    fvs.append((k,fv))
    if np.max(fv) > max_frq:
        max_frq = np.max(fv)
        if max_frq == 255.0:
            print(v)
max_frq

814.0

In [81]:
maxs = []
for k,f in fvs:
    maxs.append(np.max(f))
maxs = np.concatenate([maxs])
div = np.percentile(np.sort(maxs), 95)
div

33.0

In [82]:
normalized_fvs = []
for k,f in fvs:
    normalized_fvs.append((k, f / div))

## TODO:

* [X] Save Yeast dataset using AC and CT methods
* [ ] Do t-SNE plots of data for positive vs. negative pairs? 

In [83]:
for k, feature_vector in tqdm(normalized_fvs):
#for k,v in yeast_dset.items():
    #if os.path.exists("data/yeast_ct/{}.npy".format(k)):
    #    continue
    #feature_vector = convert_triad(v)
    #if feature_vector is None:
    #    print("{}".format(k))
    if k == 'Q9Y6Z7':
        np.save("data/human_ct/" + k + ".npy", feature_vector)

100%|██████████| 20128/20128 [00:00<00:00, 1330417.15it/s]
